In [6]:
# 21CS30064: Ishaan Sinha
# IHMCDC
# Interest Match using Complete Linkage Divisive (Top-Down) Clustering Technique

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import time

In [8]:
start_time = time.time()

# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vec1, vec2)
    # Calculate the norms of the vectors
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Check if either norm is zero to avoid division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    
    # Calculate the cosine similarity
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)
    return cosine_sim

# Define a function to initialize cluster centroids randomly
def initialize_clusters(data, k):
    # Randomly choose indices for initial centroids
    centroids_indices = np.random.choice(data.shape[0], size=k, replace=False)
    # Get the data points corresponding to the chosen indices
    centroids = data[centroids_indices]
    return centroids

# Define a function to assign data points to clusters based on similarity to centroids
def assign_to_clusters(data, centroids):
    # Initialize a matrix to store similarities between data points and centroids
    similarity_matrix = np.zeros((data.shape[0], centroids.shape[0]))
    # Calculate cosine similarity between each data point and each centroid
    for i in range(data.shape[0]):
        for j in range(centroids.shape[0]):
            similarity_matrix[i, j] = cosine_similarity(data[i], centroids[j])
    # Assign each data point to the cluster with the highest similarity
    labels = np.argmax(similarity_matrix, axis=1)
    return labels

# Define a function to update centroids based on assigned data points
def update_centroids(data, labels, k):
    # Initialize an array to store updated centroids
    centroids = np.zeros((k, data.shape[1]))
    # Iterate over each cluster
    for cluster_label in range(k):
        # Extract data points belonging to the current cluster
        cluster_data = data[labels == cluster_label]
        # Calculate the mean of the data points to obtain the new centroid
        if len(cluster_data) > 0:
            centroids[cluster_label] = np.mean(cluster_data, axis=0)
        else:
            centroids[cluster_label] = np.zeros(data.shape[1])
    return centroids

# Define a function to perform K-means clustering
def k_means_clustering(data, k, iterations):
    # Initialize centroids
    centroids = initialize_clusters(data, k)
    # Iterate for a fixed number of iterations
    for _ in range(iterations):
        # Assign data points to clusters
        labels = assign_to_clusters(data, centroids)
        # Update centroids based on assigned data points
        centroids = update_centroids(data, labels, k)
    return labels, centroids

# Define a function to calculate the silhouette coefficient for clustering quality evaluation
def silhouette_coefficient(data, labels, centroids):
    # Get the number of samples
    num_samples = len(data)
    # Initialize an array to store silhouette scores for each sample
    scores = np.zeros(num_samples)

    # Calculate silhouette score for each sample
    for i in range(num_samples):
        # Calculate distance within the same cluster (a)
        a = np.mean(np.linalg.norm(data[i] - data[labels == labels[i]], axis=1))
        # Calculate distance to the nearest cluster (b)
        b = np.min(np.mean(np.linalg.norm(data[i] - centroids, axis=1)))
        # Calculate silhouette coefficient for the current sample
        scores[i] = (b - a) / max(a, b)

    # Calculate the mean silhouette coefficient across all samples
    return np.mean(scores)

# Define a function to compute Jaccard similarity between two sets of cluster labels
def jaccard_similarity(label1, label2):
    # Initialize lists to store indices of samples belonging to each cluster in both sets
    list1 = []
    list2 = []
    # Calculate the number of clusters in each set
    len1 = len(np.unique(label1))
    len2 = len(np.unique(label2))
    
    # Iterate over each cluster in the first set
    for i in range(len1):
        temp = []
        # Find indices of samples belonging to the current cluster in the first set
        for j in range(len(label1)):
            if int(label1[j]) == i:
                temp.append(j)
        list1.append(temp)
    
    # Iterate over each cluster in the second set
    for i in range(len2):
        temp = []
        # Find indices of samples belonging to the current cluster in the second set
        for j in range(len(label2)):
            if int(label2[j]) == i:
                temp.append(j)
        list2.append(temp)

    # Initialize a list to store Jaccard similarity scores for each cluster
    jaccard_scores = []

    # Iterate over each cluster in the first set
    for i in range(len(list1)):
        # Initialize the maximum Jaccard similarity score for the current cluster
        jaccard_max = -1
        # Iterate over each cluster in the second set
        for j in range(len(list2)):
            # Calculate the intersection and union of the two clusters
            intersection = len(set(list1[i]).intersection(list2[j]))
            union = len(set(list1[i]).union(list2[j]))
            # Calculate Jaccard similarity for the current pair of clusters
            jaccard = intersection / union if union != 0 else 0
            # Update the maximum Jaccard similarity score for the current cluster
            jaccard_max = max(jaccard_max, jaccard)
        # Append the maximum Jaccard similarity score to the list
        jaccard_scores.append(jaccard_max)

    # Print the Jaccard similarity scores for each cluster in the first set
    for i in range(len(jaccard_scores)):
        print("Jaccard Similarity for ", i+1, "th cluster is ", jaccard_scores[i])

# Define a class for complete linkage divisive clustering
class CompleteLinkageDivisiveClustering:
    def __init__(self, k):
        self.k = k
        self.labels = None
    
    def fit(self, data):
        # Get the number of samples
        num_samples = len(data)
        # Initialize cluster labels
        self.labels = np.zeros(num_samples)
        
        # Continue splitting clusters until the desired number of clusters is reached
        while len(np.unique(self.labels)) < self.k:
            # Initialize a matrix to store pairwise cosine similarity between samples
            similarity_matrix  = np.zeros((data.shape[0], data.shape[0]))
            # Calculate pairwise cosine similarity between all samples
            for i in range(data.shape[0]):
                for j in range(data.shape[0]):
                    similarity_matrix[i, j] = cosine_similarity(data[i], data[j])
            # Get the current number of clusters
            num_clusters = len(np.unique(self.labels))
            # Initialize variables to track maximum similarity and cluster indices
            maxxx = -1
            indd1 = -1
            indd2 = -1
            cluster = -1
            # Iterate over each existing cluster
            for i in range(num_clusters):
                # Get indices of samples belonging to the current cluster
                cluster_indices = np.where(self.labels == i)[0]
                ind1 = -1
                ind2 = -1
                max_distance = -1
                # Iterate over each pair of samples within the current cluster
                for j in range(len(cluster_indices)):
                    for k in range(j+1, len(cluster_indices)):
                        # Find the pair with the maximum similarity
                        if similarity_matrix[cluster_indices[j], cluster_indices[k]] > max_distance:
                            max_distance = similarity_matrix[cluster_indices[j], cluster_indices[k]]
                            ind1 = cluster_indices[j]
                            ind2 = cluster_indices[k]
                # Update variables if the maximum similarity exceeds the current maximum
                if(max_distance > maxxx):
                    maxxx = max_distance
                    indd1 = ind1
                    indd2 = ind2
                    cluster = i

            # Get the label for the largest cluster after splitting
            largest = np.max(self.labels)+1
            # Get indices of samples belonging to the cluster chosen for splitting
            cluster_indices_temp = np.where(self.labels == cluster)[0]
            # Update cluster labels based on similarity between samples and split point
            for i in range(len(cluster_indices_temp)):
                if similarity_matrix[indd1, cluster_indices_temp[i]] > similarity_matrix[indd2, cluster_indices_temp[i]]:
                    self.labels[cluster_indices_temp[i]] = largest

In [10]:
if __name__ == "__main__":
    # Load the dataset
    data = pd.read_csv('interests.csv')

    # Drop the class labels if they exist
    if 'class' in data.columns:
        data = data.drop('class', axis=1)
    
    # Convert the data to a numpy array
    data = data.values
    
    # In the dataset, wherever any row contains a blank cell, replace it with 0
    data = np.nan_to_num(data)

    # Define the range of k values to test
    k_values = [3, 4, 5, 6]
    iterations = 20
    
    # Track the highest Silhouette Coefficient and its corresponding k value
    max_silhouette_score = -1
    optimal_k = None
    
    # Iterate over each k value
    for k in k_values:
        # Perform K-means clustering
        labels, centroids = k_means_clustering(data, k, iterations)
  
        # Calculate the Silhouette Coefficient
        silhouette_score = silhouette_coefficient(data, labels, centroids)
        
        # Print Silhouette Coefficient for each k value
        print(f"For k = {k}, Silhouette Coefficient: {silhouette_score}")
        
        # Update optimal k if the current Silhouette Coefficient is higher
        if silhouette_score > max_silhouette_score:
            max_silhouette_score = silhouette_score
            optimal_k = k

    # Perform K-means clustering with the optimal k value
    Labels, Centroids = k_means_clustering(data, optimal_k, iterations)
    
    print(f"\nOptimal number of clusters (k): {optimal_k} with Silhouette Coefficient: {max_silhouette_score}")

    # Perform divisive clustering
    divisive_clustering = CompleteLinkageDivisiveClustering(optimal_k)
    divisive_clustering.fit(data)
    
    divisive_labels = divisive_clustering.labels
    
    # Generate kmeans.txt
    with open('kmeans.txt', 'w') as f:
        for i in range(optimal_k):
            cluster_indices = np.where(Labels == i)[0]
            cluster_indices = sorted(cluster_indices)
            f.write(','.join(map(str, cluster_indices)) + '\n')
    
    # Generate divisive.txt
    with open('divisive.txt', 'w') as f:
        for i in range(optimal_k):
            cluster_indices = np.where(divisive_labels == i)[0]
            cluster_indices = sorted(cluster_indices)
            f.write(','.join(map(str, cluster_indices)) + '\n')

    print("Files kmeans.txt and divisive.txt generated successfully.")

    # Compute Jaccard similarity between K-means and divisive clustering results
    jaccard_similarity(Labels, divisive_labels)

print("--- %s seconds runtime---" % (time.time() - start_time))

For k = 3, Silhouette Coefficient: -0.1466375193058558
For k = 4, Silhouette Coefficient: -0.09645624017887024
For k = 5, Silhouette Coefficient: -0.061490565007007995
For k = 6, Silhouette Coefficient: -0.024205641207828095

Optimal number of clusters (k): 6 with Silhouette Coefficient: -0.024205641207828095
Files kmeans.txt and divisive.txt generated successfully.
Jaccard Similarity for  1 th cluster is  0.2346002621231979
Jaccard Similarity for  2 th cluster is  0.175
Jaccard Similarity for  3 th cluster is  0.1507537688442211
Jaccard Similarity for  4 th cluster is  0.16133333333333333
Jaccard Similarity for  5 th cluster is  0.2248995983935743
Jaccard Similarity for  6 th cluster is  0.16339869281045752
--- 410.71196603775024 seconds runtime---
